# Modifications de résultats des sondages

## Import préliminaires des librairies

In [1]:
import re
import numpy as np
import pandas as pd
import locale 
from locale import atof

# Read files
import PyPDF2
from pdfminer.high_level import extract_text
import camelot as km

### Perso ###
from toolbox import *

In [2]:
fp="rapport/2018-12-08.pdf"
text = extract_text(fp,page_numbers=[23])
question1="Êtes-vous  satisfait  ou  mécontent  d"
question2="Êtes-vous satisfait ou mécontent d"
# print(text.find(question1))
# print(text.find(question2))
# print(len(question2))
# print(re.sub('\.\.\. (?![\n])','...\n',text))

In [3]:
with open(fp,"rb") as pdf_file:
    pdf_reader= PyPDF2.PdfFileReader(pdf_file)
    isEncrypted = pdf_reader.getIsEncrypted()
    nbPages= pdf_reader.numPages

    # Define columns
    page_content=pdf_reader.getPage(15).extractText()
# print(page_content)

## Extraction utilisant une lecture de PDF

In [2]:
def extract_name(page_text):
    # Extract name
    question1="Jugementportésurl'actiond"
    question2="Queljugementportez-voussurl'actiond"
    fullname=""
    for l in page_text.split('\n'):
        if question1 in l:
            fullname=l[l.index(question1)+len(question1)+1:l.index("entantque")]
        if question2 in l:
            fullname=l[l.index(question2)+len(question2)+1:l.index("?")]
    for i in range(len(fullname)):
        if fullname[i:].isupper():
            prenom=fullname[:i]
            nom=fullname[i:]
            break
    return prenom,nom

In [3]:

def extract_ipsos_data_from_pdf(file_path,page_range,date,df=None):
    with open(file_path,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        isEncrypted = pdf_reader.getIsEncrypted()
        nbPages= pdf_reader.numPages

        # Define columns
        columns=["Nom","Categorie","Groupe","Base","Date","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]
        tables = km.read_pdf(file_path, flavor='stream',pages='{}-{}'.format(page_range[0],page_range[1]))
        template=build_one_page_template_df(tables[0].df,date)
        if df is None:
            df=pd.DataFrame(columns=columns)
        for i in range(page_range[0],page_range[1]+1):
            page_content=pdf_reader.getPage(i-1).extractText()
            nom=extract_name(page_content)
            nom=nom[0]+' '+nom[1]
            data=tables[i-page_range[0]].df
            df_one_page=build_one_page_df(nom,template,data)
            df=pd.concat([df,df_one_page],ignore_index=True)
    return df


In [22]:
filename="2021-09-04"
first_p=14
last_p=55
# df=extract_ipsos_alt("rapport/"+filename+".pdf",first_p,last_p)
df=extract_ipsos_data_from_pdf("rapport/"+filename+".pdf",(first_p,last_p),filename,None)
df


Emmanuel MACRON
Jean CASTEX
Gabriel ATTAL
Martine AUBRY
Roselyne BACHELOT
Michel BARNIER
François BAROIN
François BAYROU
Xavier BERTRAND
Jean-Michel BLANQUER
Christophe CASTANER
Bernard CAZENEUVE
Eric CIOTTI
Gérald DARMANIN
Eric DUPOND-MORETTI
Nicolas DUPONT-AIGNAN
Christian ESTROSI
Olivier FAURE
Richard FERRAND
Anne HIDALGO
François HOLLANDE
Nicolas HULOT
Christian JACOB
Yannick JADOT
Jack LANG
Gérard LARCHER
Jean-Yves LEDRIAN
Bruno LEMAIRE
Marine LEPEN
Marion MARECHAL
Jean-Luc MELENCHON
Arnaud MONTEBOURG
Valérie PECRESSE
Edouard PHILIPPE
Barbara POMPILI
Bruno RETAILLEAU
Ségolène ROYAL
François RUFFIN
Nicolas SARKOZY
Marlène SCHIAPPA
Olivier VERAN
Laurent WAUQUIEZ


,Nom,Categorie,Groupe,Base,Date,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable
0,Emmanuel MACRON,ENSEMBLE,ENSEMBLE,1000.0,2021-09-04,8.4,32.0,40.4,24.5,28.6,53.1,6.5,1.5,-2.0
1,Emmanuel MACRON,Sexe,Homme,476.0,2021-09-04,11.5,30.8,42.2,24.6,28.5,53.2,4.6,0.8,-1.8
2,Emmanuel MACRON,Sexe,Femme,524.0,2021-09-04,5.5,33.2,38.8,24.3,28.7,53.0,8.2,2.1,-2.2
3,Emmanuel MACRON,Age,18 à 24 ans,106.0,2021-09-04,10.0,40.0,50.0,21.6,20.1,41.7,8.3,-3.6,4.3
4,Emmanuel MACRON,Age,25 à 34 ans,159.0,2021-09-04,14.4,21.6,36.0,25.6,26.3,51.9,12.1,-7.3,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,Laurent WAUQUIEZ,Vote Présidentielle 2017 1er tour,ST Non expr.,250.0,2021-09-04,2.7,16.7,19.4,19.7,26.6,46.3,34.3,-4.7,1.4
2936,Laurent WAUQUIEZ,Vote Présidentielle 2017 2nd tour,Emmanuel Macron,416.0,2021-09-04,12.4,42.6,55.1,24.4,8.7,33.1,11.9,2.1,-0.9
2937,Laurent WAUQUIEZ,Vote Présidentielle 2017 2nd tour,Marine Le Pen,214.0,2021-09-04,1.8,10.3,12.1,27.4,42.8,70.3,17.7,-8.0,0.9
2938,Laurent WAUQUIEZ,Vote Présidentielle 2017 2nd tour,ST Expr.,630.0,2021-09-04,8.8,31.7,40.5,25.4,20.3,45.7,13.8,-1.3,-0.3


In [23]:
df.Nom.unique()

array(['Emmanuel MACRON', 'Jean CASTEX', 'Gabriel ATTAL', 'Martine AUBRY',
       'Roselyne BACHELOT', 'Michel BARNIER', 'François BAROIN',
       'François BAYROU', 'Xavier BERTRAND', 'Jean-Michel BLANQUER',
       'Christophe CASTANER', 'Bernard CAZENEUVE', 'Eric CIOTTI',
       'Gérald DARMANIN', 'Eric DUPOND-MORETTI', 'Nicolas DUPONT-AIGNAN',
       'Christian ESTROSI', 'Olivier FAURE', 'Richard FERRAND',
       'Anne HIDALGO', 'François HOLLANDE', 'Nicolas HULOT',
       'Christian JACOB', 'Yannick JADOT', 'Jack LANG', 'Gérard LARCHER',
       'Jean-Yves LEDRIAN', 'Bruno LEMAIRE', 'Marine LEPEN',
       'Marion MARECHAL', 'Jean-Luc MELENCHON', 'Arnaud MONTEBOURG',
       'Valérie PECRESSE', 'Edouard PHILIPPE', 'Barbara POMPILI',
       'Bruno RETAILLEAU', 'Ségolène ROYAL', 'François RUFFIN',
       'Nicolas SARKOZY', 'Marlène SCHIAPPA', 'Olivier VERAN',
       'Laurent WAUQUIEZ'], dtype=object)

In [8]:
# for n in df_alt.Nom.unique():
#     t1=df[df.Nom==n].iloc[0]
#     t2=df_alt[df_alt.Nom==n].iloc[0]
#     if t1!=t2:
#         print('PB:\n{}\n{}'.format(t1,t2))

In [9]:
# df3=pd.concat([df,df2],ignore_index=True)
# df3.to_pickle('data/'+filename+'.p')

In [24]:
# Reading IPSOS report

# with open("rapport/2020-10-14.pdf","rb") as pdf_file:
#     pdf_report= PyPDF2.PdfFileReader(pdf_file)
#     isEncrypted = pdf_report.getIsEncrypted()
#     nbPages= pdf_report.numPages

#     page1=pdf_report.getPage(15)
#     text=page1.extractText().replace(',','.')
#     text = extract_text(pdf_file,page_numbers=[14])

# print(text)

print(df.isnull().values.any())
df.head(10)
df.describe()
df[df.Base==0]

False


,Nom,Categorie,Groupe,Base,Date,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable


In [25]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()
df[df.isnull().any(axis=1)]

,Nom,Categorie,Groupe,Base,Date,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable


## Find mistakes and try to correct them

In [26]:
def find_fav_mistake(df):
    tfav=df['Très favorable']
    pfav=df['Plutôt favorable']
    stfav=df['ST Favorable']
    fav_df= df[np.abs(tfav+pfav-stfav)>0.11]
    return fav_df

def find_def_mistake(df):
    tdef=df['Très défavorable']
    pdef=df['Plutôt défavorable']
    stdef=df['ST Défavorable']
    def_df= df[np.abs(tdef+pdef-stdef)>0.11]
    return def_df

def find_nsp_mistake(df):
    stfav=df['ST Favorable']
    stdef=df['ST Défavorable']
    nsp=df['Nsp']
    tot_df= df[np.abs(stfav+stdef+nsp-100)>0.11]
    return tot_df

def find_mistake(df):
    fav_df= find_fav_mistake(df)
    def_df= find_def_mistake(df)
    tot_df=find_nsp_mistake(df)
    error_df=pd.concat([fav_df,def_df,tot_df])
    return error_df

In [27]:
def realign_fav(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très favorable","Plutôt favorable","ST Favorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax and len(zeros_cols)>0:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_fav(df,i+1)
    return df

def realign_def(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très défavorable","Plutôt défavorable","ST Défavorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_def(df,i+1)
    return df

def realign_nsp(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["ST Favorable","ST Défavorable","Nsp"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_nsp(df,i+1)
    return df

    



In [28]:
def correct_mistakes(df):
    df_err=find_fav_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_fav(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_def_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_def(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_nsp_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_nsp(df_err[cond])
    df.loc[df_err.index]=df_err
    return df

In [29]:
df_err=find_mistake(df)
df_err
# df=correct_mistakes(df)


,Nom,Categorie,Groupe,Base,Date,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable


In [16]:
df_err=find_mistake(df)
while len(df_err)>0:
    index=df_err.iloc[0:1].index[0]
    val=df_err.iloc[0].values[3:]
    print(df_err.iloc[0])
    print(val)
    correct=[]
    cols=["Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]
    while len(correct)!=10:
        correct=input()
        correct=[float(x) for x in correct.split(' ')]
    print(correct)
    df.loc[index,cols]=correct
    df_err=find_mistake(df)

In [30]:

# df.to_pickle('data/'+filename+'.p')